<a href="https://colab.research.google.com/github/ishidesigns/ML-Major/blob/main/AirBNB_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Sentiment Analysis of Airbnb reviews

In [ ]:
!pip install ipython-autotime
%load_ext autotime

time: 1.39 ms


In [ ]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 6.0MB/s 
time: 3.26 s


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()

time: 26.3 ms


In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/ML/Dataset/reviews.csv')
df

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...
...,...,...,...,...,...,...
84844,3624990,50436321,2015-10-12,37419458,Ryan,The description and pictures of the apartment ...
84845,3624990,51024875,2015-10-17,6933252,Linda,We had an excellent stay. It was clean and com...
84846,3624990,51511988,2015-10-20,19543701,Jaime,"Gran ubicación, cerca de todo lo atractivo del..."
84847,3624990,52814482,2015-11-02,24445024,Jørgen,"Very good apartement, clean and well sized. Si..."


time: 7.36 s


In [ ]:
reviews = df[['id','comments']]
reviews = reviews.applymap(str)
reviews

,id,comments
0,38917982,Cute and cozy place. Perfect location to every...
1,39087409,Kelly has a great room in a very central locat...
2,39820030,"Very spacious apartment, and in a great neighb..."
3,40813543,Close to Seattle Center and all it has to offe...
4,41986501,Kelly was a great host and very accommodating ...
...,...,...
84844,50436321,The description and pictures of the apartment ...
84845,51024875,We had an excellent stay. It was clean and com...
84846,51511988,"Gran ubicación, cerca de todo lo atractivo del..."
84847,52814482,"Very good apartement, clean and well sized. Si..."


time: 76.5 ms


In [ ]:
import nltk
nltk.download('stopwords')
stopword_list =  nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
time: 1.76 s


In [ ]:
!pip install contractions
import contractions 
def con(text):
  expand = contractions.fix(text)
  return expand

     |████████████████████████████████| 245kB 4.3MB/s 
     |████████████████████████████████| 317kB 5.9MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81696 sha256=ee713c92b3005b3e74de79ca0b34c3811f54f85e1c5cee532a7472b01b5fea6c
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
time: 7.21 s


In [ ]:
import re
def remove_sp(text):
  pattern = r'[^A-Za-z0-9\s]' 
  text = re.sub(pattern,'',text)
  return text

time: 1.74 ms


In [ ]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()

time: 1.15 ms


In [ ]:
def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

time: 3.03 ms


In [ ]:

reviews.comments = reviews.comments.apply(con)

reviews.comments = reviews.comments.apply(remove_sp)

reviews.comments = reviews.comments.apply(lambda x:x.lower())

reviews.comments = reviews.comments.apply(remove_stopwords)

reviews.head()


,id,comments
0,38917982,cute cozy place perfect location everything
1,39087409,kelly great room central location beautiful bu...
2,39820030,spacious apartment great neighborhood kind apa...
3,40813543,close seattle center offer ballet theater muse...
4,41986501,kelly great host accommodating great neighborh...


time: 22.8 s


In [ ]:
reviews['compound'] = reviews['comments'].apply(lambda x: vs.polarity_scores(x)['compound'])
reviews.head()

,id,comments,compound
0,38917982,cute cozy place perfect location everything,0.7717
1,39087409,kelly great room central location beautiful bu...,0.9867
2,39820030,spacious apartment great neighborhood kind apa...,0.9325
3,40813543,close seattle center offer ballet theater muse...,0.8313
4,41986501,kelly great host accommodating great neighborh...,0.9769


time: 42.5 s


In [ ]:
def sentiment(reviews):
    if reviews['compound'] >= 0.05:
        return 'positive'
    elif reviews['compound'] <= -0.05:
        return 'negative' 
    else:
        return 'neutral'

reviews['sentiment'] = reviews.apply(sentiment, axis=1)
reviews.head()

,id,comments,compound,sentiment
0,38917982,cute cozy place perfect location everything,0.7717,positive
1,39087409,kelly great room central location beautiful bu...,0.9867,positive
2,39820030,spacious apartment great neighborhood kind apa...,0.9325,positive
3,40813543,close seattle center offer ballet theater muse...,0.8313,positive
4,41986501,kelly great host accommodating great neighborh...,0.9769,positive


time: 880 ms


In [ ]:
reviews.to_csv('reviews_sentiment.csv',index=False)

time: 687 ms


In [ ]:
%%writefile app1.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

df = pd.read_csv('/content/reviews_sentiment.csv')
df['comments'] = df['comments'].astype('unicode')
df['sentiment'] = df['sentiment'].astype('unicode')
x = df.iloc[:,1].values # comments column as input
y = df.iloc[:,3].values # sentiment column as output
st.title("Sentiment Analysis of Airbnb Reviews")
st.subheader('TFIFD Vectorizer')

text_model = Pipeline([('tfidf',TfidfVectorizer()),('model',MultinomialNB())])
text_model.fit(x,y)
message = st.text_area("Enter Text","Type Here ..")
op = text_model.predict([message])
if st.button("Predict"):
  st.title(op)

Writing app1.py
time: 2.26 ms


In [ ]:
!pip install streamlit

     |████████████████████████████████| 7.4MB 5.9MB/s 
     |████████████████████████████████| 112kB 40.3MB/s 
     |████████████████████████████████| 163kB 38.9MB/s 
     |████████████████████████████████| 7.0MB 37.9MB/s 
     |████████████████████████████████| 102kB 9.4MB/s 
     |████████████████████████████████| 4.5MB 32.6MB/s 
     |████████████████████████████████| 133kB 33.4MB/s 
     |████████████████████████████████| 71kB 6.3MB/s 
     |████████████████████████████████| 122kB 25.9MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=cce64252808a9f1bcfc816e1fa859a70f6f7e60386df7bebb46403be54720819
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
  Created wheel for watchdog: filename=watchdog-0.10.4-cp36-none-any.whl size=74841 sha256=31ee3f7f7bffd20d1d8cd27da3abd915b1416e5a1287319d2e4b4480ff65814d
  Stored in directory: /root/.ca

time: 22.1 s


In [ ]:
!pip install pyngrok==4.1.1

  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp36-none-any.whl size=15971 sha256=815dd60075a2ad5b7d4ce36fdecca84e3f2847c88f7ad691d764ea864b8abc1a
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok
time: 4.48 s


In [ ]:
from pyngrok import ngrok
url = ngrok.connect(port='8501')
url

'http://afdf5a7aae09.ngrok.io'

time: 1.09 s


In [ ]:
!streamlit run app1.py &


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.73.111.235:8501

  Stopping...
time: 1min 21s
